## Librerías

In [2]:
# carga de datos
import pandas as pd
import io
from google.colab import files
# manipulación y visualización
import numpy as np
from google.colab import files

## Carga de excells

In [3]:
def upload_files ():
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    df = pd.read_excel(fn, sheet_name=None, dtype={'IDArticulo': np.int32, 'IDOrden': np.int32, 'Cantidad': np.int32, 'IDCliente': np.int32,'Nacimiento': np.int32, 'IDFactura': np.int32}) 
    return df

In [4]:
Datos_Entrada = upload_files()

Saving miTiendaOnline.xlsx to miTiendaOnline.xlsx
User uploaded file "miTiendaOnline.xlsx" with length 15327 bytes


# DataSet lista: Lista productos

In [ ]:
Articulos = Datos_Entrada['Articulos'][['Nombre']].copy()
Articulos.head()

,Nombre
0,Gigabyte AORUS 7 KB-7ES1130SD Intel Core i7-10...
1,Samsung Chromebook 4 XE310XBA-K01ES Intel Cele...
2,Asus TUF Gaming F15 FX506LH-BQ116 Intel Core i...
3,"Lenovo Tab M10 10.1"" HD 2/32GB Negra"
4,"Samsung Galaxy Tab A7 10.4"" 32GB Wifi Plata"


In [ ]:
with open('ListaArticulos.csv', 'w') as f:
  Articulos.to_csv(f,  index = False)
  
files.download('ListaArticulos.csv')

# DataSet KPI: Facturación total

In [ ]:
Articulos = Datos_Entrada['Articulos'][['IDArticulo', 'PrecioVenta']].copy()
Articulos.head()

,IDArticulo,PrecioVenta
0,1,1159.0
1,2,299.0
2,3,1181.0
3,4,139.0
4,5,199.0


In [ ]:
Ordenes = Datos_Entrada['Ordenes'][['IDOrden', 'IDFactura']].copy()
# Eliminamos aquellas ordenes que no tengan factura
filtro = Ordenes['IDFactura'] > 0
Ordenes = Ordenes[filtro]
Ordenes = Ordenes.drop(['IDFactura'], axis=1)
Ordenes.head()

,IDOrden
0,1
1,2
2,3
3,4
5,6


In [ ]:
Detalles = Datos_Entrada['Detalles'][['IDOrden', 'IDArticulo', 'Cantidad']].copy()
Detalles.head()

,IDOrden,IDArticulo,Cantidad
0,1,1,1
1,2,3,1
2,2,2,1
3,3,3,1
4,3,1,1


In [ ]:
df = pd.merge(Ordenes, Detalles, on='IDOrden', how='left')
df1 = pd.merge(df, Articulos, on='IDArticulo', how='left')
df2 = df1[['IDArticulo','Cantidad']].groupby(by=["IDArticulo"]).sum('Cantidad')
df3 = pd.merge(df2, Articulos, on='IDArticulo', how='left')
df3.insert(1, 'Total', df3['Cantidad']*df3['PrecioVenta'])
df3.head()

,IDArticulo,Total,Cantidad,PrecioVenta
0,1,24339.0,21,1159.0
1,2,5083.0,17,299.0
2,3,10629.0,9,1181.0
3,4,1946.0,14,139.0
4,5,2388.0,12,199.0


In [ ]:
with open('FacturacionTotal.csv', 'w') as f:
  df3.to_csv(f,  index = False)
  
files.download('FacturacionTotal.csv')

# DataSet Histograma: Ventas

In [22]:
Articulos = Datos_Entrada['Articulos'][['IDArticulo', 'PrecioVenta','Nombre']].copy()
Articulos.head()

,IDArticulo,PrecioVenta,Nombre
0,1,1159.0,Gigabyte AORUS 7 KB-7ES1130SD Intel Core i7-10...
1,2,299.0,Samsung Chromebook 4 XE310XBA-K01ES Intel Cele...
2,3,1181.0,Asus TUF Gaming F15 FX506LH-BQ116 Intel Core i...
3,4,139.0,"Lenovo Tab M10 10.1"" HD 2/32GB Negra"
4,5,199.0,"Samsung Galaxy Tab A7 10.4"" 32GB Wifi Plata"


In [23]:
Ordenes = Datos_Entrada['Ordenes'][['IDOrden', 'IDFactura']].copy()
# Eliminamos aquellas ordenes que no tengan factura
filtro = Ordenes['IDFactura'] > 0
Ordenes = Ordenes[filtro]
Ordenes = Ordenes.drop(['IDFactura'], axis=1)
Ordenes.head()

,IDOrden
0,1
1,2
2,3
3,4
5,6


In [24]:
Facturas = Datos_Entrada['Facturas'][['IDOrden', 'FechaFactura']].copy()
Facturas.head()

,IDOrden,FechaFactura
0,1,2022-01-02
1,2,2022-01-03
2,3,2022-01-05
3,4,2022-01-05
4,6,2022-01-06


In [25]:
Detalles = Datos_Entrada['Detalles'][['IDOrden', 'IDArticulo', 'Cantidad']].copy()
Detalles.head()

,IDOrden,IDArticulo,Cantidad
0,1,1,1
1,2,3,1
2,2,2,1
3,3,3,1
4,3,1,1


In [45]:
df = pd.merge(Facturas, Ordenes, on='IDOrden', how='left')
df1 = pd.merge(df, Detalles, on='IDOrden', how='left')
df2 = pd.merge(df1, Articulos, on='IDArticulo', how='left')
df3 = df2[['FechaFactura','IDArticulo','Cantidad','PrecioVenta']].groupby(by=['FechaFactura','IDArticulo']).sum('Cantidad')
df3 = df3.astype(int)
df3=df3.reset_index()
Nombres = Articulos[['IDArticulo','Nombre']].copy()
df4 = pd.merge(df3, Nombres, on='IDArticulo', how='left')
df4.head()

,FechaFactura,IDArticulo,Cantidad,PrecioVenta,Nombre
0,2022-01-02,1,1,1159,Gigabyte AORUS 7 KB-7ES1130SD Intel Core i7-10...
1,2022-01-03,2,1,299,Samsung Chromebook 4 XE310XBA-K01ES Intel Cele...
2,2022-01-03,3,1,1181,Asus TUF Gaming F15 FX506LH-BQ116 Intel Core i...
3,2022-01-05,1,2,2318,Gigabyte AORUS 7 KB-7ES1130SD Intel Core i7-10...
4,2022-01-05,2,2,299,Samsung Chromebook 4 XE310XBA-K01ES Intel Cele...


In [47]:
from google.colab import files

with open('HistogramaVenta.csv', 'w') as f:
  df4.to_csv(f,  index = True)
  
files.download('HistogramaVenta.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# DataSet Tarta: Categoria

In [4]:
Articulos = Datos_Entrada['Articulos'][['IDArticulo', 'Categoria']].copy()
Articulos.head()

,IDArticulo,Categoria
0,1,Portatil
1,2,Portatil
2,3,Portatil
3,4,Tableta
4,5,Tableta


In [5]:
Ordenes = Datos_Entrada['Ordenes'][['IDOrden', 'IDFactura']].copy()
# Eliminamos aquellas ordenes que no tengan factura
filtro = Ordenes['IDFactura'] > 0
Ordenes = Ordenes[filtro]
Ordenes = Ordenes.drop(['IDFactura'], axis=1)
Ordenes.head()

,IDOrden
0,1
1,2
2,3
3,4
5,6


In [6]:
Detalles = Datos_Entrada['Detalles'][['IDOrden', 'IDArticulo', 'Cantidad']].copy()
Detalles.head()

,IDOrden,IDArticulo,Cantidad
0,1,1,1
1,2,3,1
2,2,2,1
3,3,3,1
4,3,1,1


In [7]:
df = pd.merge(Ordenes, Detalles, on='IDOrden', how='left')
df1 = pd.merge(df, Articulos, on='IDArticulo', how='left')
df2 = df1[['IDArticulo', 'Cantidad']].groupby(by=["IDArticulo"]).sum('Cantidad')
df3 = pd.merge(df2, Articulos, on='IDArticulo', how='left')
df4 = df3[['Categoria', 'Cantidad']].groupby(by=["Categoria"]).sum('Cantidad')
df4.head()

,Cantidad
Categoria,
Portatil,47
Tableta,26


In [9]:
with open('TartaCategoria.csv', 'w') as f:
  df4.to_csv(f,  index = True)
  
files.download('TartaCategoria.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# DataSet KPI: Importe Stock

In [13]:
Articulos = Datos_Entrada['Articulos'][['IDArticulo', 'Unidades','PrecioCoste']].copy()
Articulos.head()

,IDArticulo,Unidades,PrecioCoste
0,1,10.0,750.0
1,2,8.0,130.0
2,3,8.0,710.0
3,4,15.0,85.0
4,5,17.0,105.0


In [14]:
Articulos.insert(1, 'Total', Articulos['Unidades']*Articulos['PrecioCoste'])
Articulos.head()

,IDArticulo,Total,Unidades,PrecioCoste
0,1,7500.0,10.0,750.0
1,2,1040.0,8.0,130.0
2,3,5680.0,8.0,710.0
3,4,1275.0,15.0,85.0
4,5,1785.0,17.0,105.0


In [17]:
with open('ImporteStock.csv', 'w') as f:
  Articulos.to_csv(f,  index = False)
  
files.download('ImporteStock.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# DataSet KPI: Pendiente de facturar

In [18]:
Articulos = Datos_Entrada['Articulos'][['IDArticulo', 'PrecioVenta']].copy()
Articulos.head()

,IDArticulo,PrecioVenta
0,1,1159.0
1,2,299.0
2,3,1181.0
3,4,139.0
4,5,199.0


In [19]:
Ordenes = Datos_Entrada['Ordenes'][['IDOrden', 'IDFactura']].copy()
# Eliminamos aquellas ordenes que tengan factura
filtro = Ordenes['IDFactura'] == 0
Ordenes = Ordenes[filtro]
Ordenes = Ordenes.drop(['IDFactura'], axis=1)
Ordenes.head()

,IDOrden
4,5
10,11
13,14
22,23
25,26


In [20]:
Detalles = Datos_Entrada['Detalles'][['IDOrden', 'IDArticulo', 'Cantidad']].copy()
Detalles.head()

,IDOrden,IDArticulo,Cantidad
0,1,1,1
1,2,3,1
2,2,2,1
3,3,3,1
4,3,1,1


In [21]:
df = pd.merge(Ordenes, Detalles, on='IDOrden', how='left')
df1 = pd.merge(df, Articulos, on='IDArticulo', how='left')
df2 = df1[['IDArticulo','Cantidad']].groupby(by=["IDArticulo"]).sum('Cantidad')
df3 = pd.merge(df2, Articulos, on='IDArticulo', how='left')
df3.insert(1, 'Total', df3['Cantidad']*df3['PrecioVenta'])
df3.head()

,IDArticulo,Total,Cantidad,PrecioVenta
0,1,2318.0,2,1159.0
1,2,598.0,2,299.0
2,3,2362.0,2,1181.0
3,4,278.0,2,139.0
4,5,199.0,1,199.0


In [22]:
with open('PendienteFacturar.csv', 'w') as f:
  df3.to_csv(f,  index = False)
  
files.download('PendienteFacturar.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# DataSet KPI: Beneficio

In [23]:
Articulos = Datos_Entrada['Articulos'][['IDArticulo', 'PrecioVenta','PrecioCoste','Envio','Gestion','Almacenamiento']].copy()
Articulos.head()

,IDArticulo,PrecioVenta,PrecioCoste,Envio,Gestion,Almacenamiento
0,1,1159.0,750.0,3.0,2.0,10.0
1,2,299.0,130.0,3.0,2.0,10.0
2,3,1181.0,710.0,3.0,2.0,10.0
3,4,139.0,85.0,3.0,2.0,10.0
4,5,199.0,105.0,3.0,2.0,10.0


In [24]:
Ordenes = Datos_Entrada['Ordenes'][['IDOrden', 'IDFactura']].copy()
# Eliminamos aquellas ordenes que no tengan factura
filtro = Ordenes['IDFactura'] > 0
Ordenes = Ordenes[filtro]
Ordenes = Ordenes.drop(['IDFactura'], axis=1)
Ordenes.head()

,IDOrden
0,1
1,2
2,3
3,4
5,6


In [25]:
Detalles = Datos_Entrada['Detalles'][['IDOrden', 'IDArticulo', 'Cantidad']].copy()
Detalles.head()

,IDOrden,IDArticulo,Cantidad
0,1,1,1
1,2,3,1
2,2,2,1
3,3,3,1
4,3,1,1


In [26]:
df = pd.merge(Ordenes, Detalles, on='IDOrden', how='left')
df1 = pd.merge(df, Articulos, on='IDArticulo', how='left')
df2 = df1[['IDArticulo','Cantidad']].groupby(by=["IDArticulo"]).sum('Cantidad')
df3 = pd.merge(df2, Articulos, on='IDArticulo', how='left')
df3.insert(1, 'BeneficioNeto', df3['Cantidad']*(df3['PrecioVenta']-df3['PrecioCoste']-df3['Envio']-df3['Gestion']-df3['Almacenamiento']))
df3.insert(1, 'Facturacion', df3['Cantidad']*df3['PrecioVenta'])
df3.insert(3, 'PorcentajeBeneficio', df3['BeneficioNeto']/df3['Facturacion'])
df3.head()

,IDArticulo,Facturacion,BeneficioNeto,PorcentajeBeneficio,Cantidad,PrecioVenta,PrecioCoste,Envio,Gestion,Almacenamiento
0,1,24339.0,8274.0,0.339948,21,1159.0,750.0,3.0,2.0,10.0
1,2,5083.0,2618.0,0.515050,17,299.0,130.0,3.0,2.0,10.0
2,3,10629.0,4104.0,0.386113,9,1181.0,710.0,3.0,2.0,10.0
3,4,1946.0,546.0,0.280576,14,139.0,85.0,3.0,2.0,10.0
4,5,2388.0,948.0,0.396985,12,199.0,105.0,3.0,2.0,10.0


In [27]:
with open('KPIBeneficio.csv', 'w') as f:
  df3.to_csv(f,  index = False)
  
files.download('KPIBeneficio.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# DataSet Gráfica de barras: Venta por productos

In [33]:
Articulos = Datos_Entrada['Articulos'][['IDArticulo', 'Nombre']].copy()
Articulos.head()

,IDArticulo,Nombre
0,1,Gigabyte AORUS 7 KB-7ES1130SD Intel Core i7-10...
1,2,Samsung Chromebook 4 XE310XBA-K01ES Intel Cele...
2,3,Asus TUF Gaming F15 FX506LH-BQ116 Intel Core i...
3,4,"Lenovo Tab M10 10.1"" HD 2/32GB Negra"
4,5,"Samsung Galaxy Tab A7 10.4"" 32GB Wifi Plata"


In [34]:
Ordenes = Datos_Entrada['Ordenes'][['IDOrden', 'IDFactura']].copy()
# Eliminamos aquellas ordenes que no tengan factura
filtro = Ordenes['IDFactura'] > 0
Ordenes = Ordenes[filtro]
Ordenes = Ordenes.drop(['IDFactura'], axis=1)
Ordenes.head()

,IDOrden
0,1
1,2
2,3
3,4
5,6


In [35]:
Detalles = Datos_Entrada['Detalles'][['IDOrden', 'IDArticulo', 'Cantidad']].copy()
Detalles.head()

,IDOrden,IDArticulo,Cantidad
0,1,1,1
1,2,3,1
2,2,2,1
3,3,3,1
4,3,1,1


In [36]:
df = pd.merge(Ordenes, Detalles, on='IDOrden', how='left')
df1 = pd.merge(df, Articulos, on='IDArticulo', how='left')
df2 = df1[['IDArticulo','Cantidad']].groupby(by=["IDArticulo"]).sum('Cantidad')
df3 = pd.merge(df2, Articulos, on='IDArticulo', how='left')
df3

,IDArticulo,Cantidad,Nombre
0,1,21,Gigabyte AORUS 7 KB-7ES1130SD Intel Core i7-10...
1,2,17,Samsung Chromebook 4 XE310XBA-K01ES Intel Cele...
2,3,9,Asus TUF Gaming F15 FX506LH-BQ116 Intel Core i...
3,4,14,"Lenovo Tab M10 10.1"" HD 2/32GB Negra"
4,5,12,"Samsung Galaxy Tab A7 10.4"" 32GB Wifi Plata"


In [37]:
with open('VentasPorProducto.csv', 'w') as f:
  df3.to_csv(f,  index = False)
  
files.download('VentasPorProducto.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# DataSet Gráfica de barras: Ventas clientes por edad

In [95]:
Ordenes = Datos_Entrada['Ordenes'][['IDOrden', 'IDFactura','IDCliente']].copy()
# Eliminamos aquellas ordenes que no tengan factura
#filtro = Ordenes['IDFactura'] > 0
#Ordenes = Ordenes[filtro]
Ordenes = Ordenes.drop(['IDFactura'], axis=1)
Ordenes.head()

,IDOrden,IDCliente
0,1,1
1,2,3
2,3,2
3,4,4
4,5,10


In [96]:
Clientes = Datos_Entrada['Clientes'][['IDCliente','Genero','Nacimiento']].copy()
len(Clientes)

29

In [97]:
df = pd.merge(Ordenes, Clientes, on='IDCliente', how='left')
len(Ordenes)

39

In [98]:
dl = df.groupby('IDCliente').nunique('IDCliente')
print(len(dl))
dl.head()

25


,IDOrden,Genero,Nacimiento
IDCliente,,,
1,1,1,1
2,1,1,1
3,3,1,1
4,1,1,1
5,2,1,1


In [101]:
# Hombre
subset_df = df[df["Genero"] == 'Hombre'].groupby(by=["Nacimiento"]).count()
subset_df.insert(3, 'Hombres', subset_df['Genero'])
df_h = subset_df[['Hombres']].copy()
df_h=df_h.reset_index()
print(len(df_h))
df_h

10


,Nacimiento,Hombres
0,1988,1
1,1989,3
2,1991,2
3,1992,2
4,1993,2
5,1995,4
6,1997,2
7,2000,3
8,2001,2
9,2003,2


In [100]:
# Mujer
subset_df = df[df["Genero"] == 'Mujer'].groupby(by=["Nacimiento"]).count()
subset_df.insert(3, 'Mujeres', subset_df['Genero'])
df_m = subset_df[['Mujeres']].copy()
df_m=df_m.reset_index()
print(len(df_m))
df_m

8


,Nacimiento,Mujeres
0,1985,1
1,1991,5
2,1995,1
3,1996,3
4,1997,1
5,2000,1
6,2001,2
7,2004,2


In [112]:
#df3 = pd.merge(df2, Articulos, on='IDArticulo', how='left')
df2 = pd.merge(df_m, df_h, on='Nacimiento',how='outer')
df2 = df2.fillna(0)
df2 = df2.astype(int)
print(len(df2))
df2

13


,Nacimiento,Mujeres,Hombres
0,1985,1,0
1,1991,5,2
2,1995,1,4
3,1996,3,0
4,1997,1,2
5,2000,1,3
6,2001,2,2
7,2004,2,0
8,1988,0,1
9,1989,0,3


In [113]:
with open('VentasClientesPorEdad.csv', 'w') as f:
  df2.to_csv(f,  index = False)
  
files.download('VentasClientesPorEdad.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Datset Mapa: Ventas por zona

In [120]:
Ordenes = Datos_Entrada['Ordenes'][['IDFactura','IDCliente']].copy()
# Eliminamos aquellas ordenes que no tengan factura
filtro = Ordenes['IDFactura'] > 0
Ordenes = Ordenes[filtro]
Ordenes.head()

,IDFactura,IDCliente
0,1,1
1,2,3
2,3,2
3,4,4
5,5,7


In [121]:
Clientes = Datos_Entrada['Clientes'][['IDCliente','Pais']].copy()
Clientes.head()

,IDCliente,Pais
0,1,US
1,2,FR
2,3,MC
3,4,FR
4,5,GB


In [127]:
df = pd.merge(Ordenes, Clientes, on='IDCliente', how='left')
df1 = df[['Pais','IDFactura']].groupby(by=["Pais"]).count()
df1=df1.reset_index()
df1.insert(1, 'Ventas', df1['IDFactura'])
df1 = df1.drop(['IDFactura'], axis=1)
df1

,Pais,Ventas
0,AU,1
1,DE,3
2,DK,4
3,ES,1
4,FR,3
5,GB,3
6,IE,1
7,IT,1
8,MC,4
9,US,12


In [129]:
with open('VentasPorZona.csv', 'w') as f:
  df1.to_csv(f,  index = False)
  
files.download('VentasPorZona.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>